In [ ]:
from processors.update_tokens import main as update_tokens_main
import sys
from db.connection import DatabaseConnection
from db.queries import DatabaseQueries
from config import load_config
import logging
import random
import requests
from time import sleep
import pandas as pd


# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("token_update.log"),
        logging.StreamHandler(sys.stdout),
    ],
)
logger = logging.getLogger(__name__)

In [ ]:
# Load configuration
logger.info("Loading configuration...")
config = load_config()

# Initialize database connection and queries
logger.info("Initializing database connection...")
db_connection = DatabaseConnection(config.db)
db_queries = DatabaseQueries(db_connection)

In [ ]:
logger.info("Calling update_tokens.main()...")
update_tokens_main()

# Get recent transactions
logger.info("Fetching recent transactions...")
hourly_transactions_df = db_queries.get_recent_transactions(n=4)

# Extract unique token addresses
logger.info("Extracting unique token addresses from transactions...")
token_columns = [
    "open_input_mint",
    "open_output_mint",
    "close_input_mint",
    "close_output_mint",
]
unique_tokens = pd.unique(hourly_transactions_df[token_columns].values.ravel())
logger.info(f"Found {len(unique_tokens)} unique token addresses.")

In [ ]:
unique_tokens

In [ ]:
def fetch_token_details(chain_id, token_addresses, api_url):
    """
    Fetch token details from Dexscreener API in batches and parse the response into a DataFrame.

    Args:
        chain_id (str): The chain ID (e.g., "solana").
        token_addresses (list): List of token addresses.
        api_url (str): Base URL of the Dexscreener API.

    Returns:
        pd.DataFrame: DataFrame containing token details.
    """
    token_details_list = []

    # Filter out None values from token_addresses
    token_addresses = [address for address in token_addresses if address is not None]

    # Process addresses in batches of 30
    for i in range(0, len(token_addresses), 30):
        batch_addresses = token_addresses[i:i+30]
        try:
            response = requests.get(f"{api_url}/tokens/v1/{chain_id}/{','.join(batch_addresses)}")
            response.raise_for_status()
            data = response.json()

            # Parse each token's details in the batch
            for token_data in data:
                token_details = {
                    'chainId': token_data.get('chainId', ''),
                    'dexId': token_data.get('dexId', ''),
                    'url': token_data.get('url', ''),
                    'pairAddress': token_data.get('pairAddress', ''),
                    'labels': token_data.get('labels', []),
                    'baseToken_address': token_data.get('baseToken', {}).get('address', ''),
                    'baseToken_name': token_data.get('baseToken', {}).get('name', ''),
                    'baseToken_symbol': token_data.get('baseToken', {}).get('symbol', ''),
                    'quoteToken_address': token_data.get('quoteToken', {}).get('address', ''),
                    'quoteToken_name': token_data.get('quoteToken', {}).get('name', ''),
                    'quoteToken_symbol': token_data.get('quoteToken', {}).get('symbol', ''),
                    'priceNative': token_data.get('priceNative', 0.0),
                    'priceUsd': token_data.get('priceUsd', 0.0),
                    'txns_m5_buys': token_data.get('txns', {}).get('m5', {}).get('buys', 0),
                    'txns_m5_sells': token_data.get('txns', {}).get('m5', {}).get('sells', 0),
                    'txns_h1_buys': token_data.get('txns', {}).get('h1', {}).get('buys', 0),
                    'txns_h1_sells': token_data.get('txns', {}).get('h1', {}).get('sells', 0),
                    'txns_h6_buys': token_data.get('txns', {}).get('h6', {}).get('buys', 0),
                    'txns_h6_sells': token_data.get('txns', {}).get('h6', {}).get('sells', 0),
                    'txns_h24_buys': token_data.get('txns', {}).get('h24', {}).get('buys', 0),
                    'txns_h24_sells': token_data.get('txns', {}).get('h24', {}).get('sells', 0),
                    'volume_h24': token_data.get('volume', {}).get('h24', 0.0),
                    'volume_h6': token_data.get('volume', {}).get('h6', 0.0),
                    'volume_h1': token_data.get('volume', {}).get('h1', 0.0),
                    'volume_m5': token_data.get('volume', {}).get('m5', 0.0),
                    'priceChange_m5': token_data.get('priceChange', {}).get('m5', 0.0),
                    'priceChange_h1': token_data.get('priceChange', {}).get('h1', 0.0),
                    'priceChange_h6': token_data.get('priceChange', {}).get('h6', 0.0),
                    'priceChange_h24': token_data.get('priceChange', {}).get('h24', 0.0),
                    'liquidity_usd': token_data.get('liquidity', {}).get('usd', 0.0),
                    'liquidity_base': token_data.get('liquidity', {}).get('base', 0.0),
                    'liquidity_quote': token_data.get('liquidity', {}).get('quote', 0.0),
                    'pairCreatedAt': token_data.get('pairCreatedAt', '')
                }
                token_details_list.append(token_details)
            sleep(1)
        except requests.RequestException as e:
            print(f"Error fetching details for batch {batch_addresses}: {e}")
            sleep(1)

    # Convert list of token details to DataFrame
    token_details = pd.DataFrame(token_details_list)
    return token_details

In [ ]:
# Example usage
chain_id = "solana"
token_addresses = unique_tokens.copy()  # Replace with actual token addresses
api_url = "https://api.dexscreener.com"

token_details_df = fetch_token_details(chain_id, token_addresses, api_url)
print(token_details_df)

In [ ]:
token_details_df

In [ ]:

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# BirdEye API base URL
BIRDEYE_API_URL = "https://public-api.birdeye.so/defi/historical-price/unix"

# Flag to determine whether to use the mock API
USE_MOCK_API = True

def fetch_historical_price(token_mint, timestamp, api_key):
    """
    Fetch historical price for a given token at a specific timestamp.
    If USE_MOCK_API is True, return a random number between 0.00001 and 2.00000.
    Otherwise, call the actual BirdEye API.
    """
    if USE_MOCK_API:
        # Generate a random price between 0.00001 and 2.00000, rounded to 5 decimal places
        random_price = round(random.uniform(0.00001, 2.00000), 5)
        logger.info(f"Mock fetch for {token_mint} at {timestamp}: {random_price}")
        return random_price
    else:
        params = {"address": token_mint, "timestamp": timestamp, "apikey": api_key}
        try:
            response = requests.get(BIRDEYE_API_URL, params=params)
            response.raise_for_status()
            data = response.json()
            return data.get("price", None)
        except requests.RequestException as e:
            logger.error(f"Error fetching price for {token_mint} at {timestamp}: {e}")
            return None

def main():
    # Assume load_config, DatabaseConnection, and DatabaseQueries are defined elsewhere
    config = load_config()
    db_connection = DatabaseConnection(config.db)
    db_queries = DatabaseQueries(db_connection)

    try:
        # Fetch recent transactions
        logger.info("Fetching recent transactions...")
        transactions_df = db_queries.get_recent_transactions(n=2)

        # Add new columns for prices
        transactions_df["open_input_price"] = None
        transactions_df["close_output_price"] = None

        # Fetch historical prices (using the mocked API when USE_MOCK_API is True)
        for index, row in transactions_df.iterrows():
            if row["open_input_mint"]:
                transactions_df.at[index, "open_input_price"] = fetch_historical_price(
                    row["open_input_mint"], row["open_block_time"], config.birdeye.api_key
                )
            if row["close_output_mint"]:
                transactions_df.at[index, "close_output_price"] = fetch_historical_price(
                    row["close_output_mint"], row["close_block_time"], config.birdeye.api_key
                )
            sleep(0.2)  # Avoid hitting rate limits (if not in mock mode)

        # Extract unique token addresses
        unique_tokens = pd.unique(
            transactions_df[["open_input_mint", "open_output_mint", "close_input_mint", "close_output_mint"]].values.ravel()
        )
        unique_tokens = [token for token in unique_tokens if pd.notnull(token)]

        # Fetch token details from database
        logger.info("Fetching token details...")
        token_info_df = db_queries.get_token_info(unique_tokens)

        # Merge transactions with token info
        transactions_df = transactions_df.merge(token_info_df, left_on="open_input_mint", right_on="token", how="left")
        transactions_df = transactions_df.rename(columns={
            "decimals": "open_input_decimals",
            "creation_time": "open_input_creation_time",
            "is_suspicious": "open_input_is_suspicious"
        })

        transactions_df = transactions_df.merge(token_info_df, left_on="close_output_mint", right_on="token", how="left", suffixes=("", "_close"))
        transactions_df = transactions_df.rename(columns={
            "decimals_close": "close_output_decimals",
            "creation_time_close": "close_output_creation_time",
            "is_suspicious_close": "close_output_is_suspicious"
        })

        # Save the final dataframe
        output_file = "updated_transactions.xlsx"
        transactions_df.to_excel(output_file, index=False)
        logger.info(f"Updated transactions saved to {output_file}")

    except Exception as e:
        logger.error(f"An error occurred: {e}")
    finally:
        db_connection.close()

if __name__ == "__main__":
    main()


In [8]:
import logging
import requests
import pandas as pd
from time import sleep
from tqdm import tqdm  # progress bar package
from processors.update_tokens import main as update_tokens_main
import sys
from db.connection import DatabaseConnection
from db.queries import DatabaseQueries
from config import load_config
import logging
import random
import requests
from time import sleep
import pandas as pd

# Configure logging globally
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# -------------------------------------------------------------------
# Updated fetch_historical_price function (as previously defined)
def fetch_historical_price(token_mint, timestamp, api_key):
    """
    Fetch the historical price for a given token at a specific timestamp using the Birdeye API.
    Constructs the URL with the required query parameters (address and unixtime) and headers.

    Expected API response schema:
    {
      "success": true,
      "data": {
        "value": 128.09276765626564,
        "updateUnixTime": 1726675897,
        "priceChange24h": -4.924324221890145
      }
    }

    Returns the 'value' (price) if successful, otherwise None.
    """
    try:
        # Ensure timestamp is an integer
        timestamp_int = int(timestamp)
    except Exception as e:
        logger.error(f"fetch_historical_price: Could not convert timestamp '{timestamp}' to int: {e}")
        return None

    # Construct URL with required parameters
    url = f"https://public-api.birdeye.so/defi/historical_price_unix?address={token_mint}&unixtime={timestamp_int}"
    logger.info(f"fetch_historical_price: Requesting price for token '{token_mint}' at timestamp '{timestamp_int}'.")

    headers = {
        "accept": "application/json",
        "x-chain": "solana",
        "X-API-KEY": api_key
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        if data.get("success") is True:
            token_data = data.get("data")
            if token_data is not None:
                price = token_data.get("value")
                logger.info(f"fetch_historical_price: Received price {price} for token '{token_mint}' at timestamp '{timestamp_int}'.")
                return price
            else:
                logger.error(f"fetch_historical_price: API returned success but data is None for token '{token_mint}' at timestamp '{timestamp_int}'. Response: {data}")
                return None
        else:
            logger.error(f"fetch_historical_price: API returned error for token '{token_mint}' at timestamp '{timestamp_int}'. Response: {data}")
            return None
    except requests.RequestException as e:
        logger.error(f"fetch_historical_price: Request error for token '{token_mint}' at timestamp '{timestamp_int}': {e}")
        return None

# -------------------------------------------------------------------
def fetch_token_details(chain_id, token_addresses, api_url):
    """
    Fetch token details from the Dexscreener API in batches and return a DataFrame.

    Processes token addresses in batches of 30 and uses a progress bar for visibility.

    The function now also extracts the 'fdv' and 'marketCap' fields from the API response.

    Example API response for each token:
    [
      {
        "chainId": "solana",
        "dexId": "meteora",
        "url": "https://dexscreener.com/solana/...",
        "pairAddress": "BwE7MRPusY...",
        "labels": ["DLMM"],
        "baseToken": {
          "address": "2RBko3xoz56aH69isQMUpzZd9NYHahhwC23A5F3Spkin",
          "name": "PUMPKIN",
          "symbol": "PKIN"
        },
        "quoteToken": {
          "address": "So11111111111111111111111111111111111111112",
          "name": "Wrapped SOL",
          "symbol": "SOL"
        },
        "priceNative": "0.00006499",
        "priceUsd": "0.01328",
        "txns": { ... },
        "volume": { ... },
        "priceChange": { ... },
        "liquidity": { "usd": 19529.75, "base": 829331, "quote": 41.6679 },
        "fdv": 13281688,
        "marketCap": 12484039,
        "pairCreatedAt": 1738140160000,
        "info": { ... }
      }
    ]

    Returns:
        pd.DataFrame: A DataFrame containing token details, including the new 'fdv' and 'marketCap' fields.
    """
    logger.info(f"fetch_token_details: Starting to fetch details for {len(token_addresses)} tokens on chain '{chain_id}'.")
    token_details_list = []

    # Filter out any None values from token_addresses
    token_addresses = [address for address in token_addresses if address is not None]

    # Process addresses in batches of 30 with a progress bar
    total_batches = (len(token_addresses) + 29) // 30
    for i in tqdm(range(0, len(token_addresses), 30), desc='Processing token batches', unit='batch', total=total_batches):
        batch_addresses = token_addresses[i:i+30]
        logger.info(f"fetch_token_details: Processing batch {i//30 + 1} with tokens: {batch_addresses}")
        try:
            response = requests.get(f"{api_url}/tokens/v1/{chain_id}/{','.join(batch_addresses)}")
            response.raise_for_status()
            data = response.json()

            for token_data in data:
                token_details = {
                    'chainId': token_data.get('chainId', ''),
                    'dexId': token_data.get('dexId', ''),
                    'url': token_data.get('url', ''),
                    'pairAddress': token_data.get('pairAddress', ''),
                    'labels': token_data.get('labels', []),
                    'baseToken_address': token_data.get('baseToken', {}).get('address', ''),
                    'baseToken_name': token_data.get('baseToken', {}).get('name', ''),
                    'baseToken_symbol': token_data.get('baseToken', {}).get('symbol', ''),
                    'quoteToken_address': token_data.get('quoteToken', {}).get('address', ''),
                    'quoteToken_name': token_data.get('quoteToken', {}).get('name', ''),
                    'quoteToken_symbol': token_data.get('quoteToken', {}).get('symbol', ''),
                    'priceNative': token_data.get('priceNative', 0.0),
                    'priceUsd': token_data.get('priceUsd', 0.0),
                    'txns_m5_buys': token_data.get('txns', {}).get('m5', {}).get('buys', 0),
                    'txns_m5_sells': token_data.get('txns', {}).get('m5', {}).get('sells', 0),
                    'txns_h1_buys': token_data.get('txns', {}).get('h1', {}).get('buys', 0),
                    'txns_h1_sells': token_data.get('txns', {}).get('h1', {}).get('sells', 0),
                    'txns_h6_buys': token_data.get('txns', {}).get('h6', {}).get('buys', 0),
                    'txns_h6_sells': token_data.get('txns', {}).get('h6', {}).get('sells', 0),
                    'txns_h24_buys': token_data.get('txns', {}).get('h24', {}).get('buys', 0),
                    'txns_h24_sells': token_data.get('txns', {}).get('h24', {}).get('sells', 0),
                    'volume_h24': token_data.get('volume', {}).get('h24', 0.0),
                    'volume_h6': token_data.get('volume', {}).get('h6', 0.0),
                    'volume_h1': token_data.get('volume', {}).get('h1', 0.0),
                    'volume_m5': token_data.get('volume', {}).get('m5', 0.0),
                    'priceChange_m5': token_data.get('priceChange', {}).get('m5', 0.0),
                    'priceChange_h1': token_data.get('priceChange', {}).get('h1', 0.0),
                    'priceChange_h6': token_data.get('priceChange', {}).get('h6', 0.0),
                    'priceChange_h24': token_data.get('priceChange', {}).get('h24', 0.0),
                    'liquidity_usd': token_data.get('liquidity', {}).get('usd', 0.0),
                    'liquidity_base': token_data.get('liquidity', {}).get('base', 0.0),
                    'liquidity_quote': token_data.get('liquidity', {}).get('quote', 0.0),
                    # New fields:
                    'fdv': token_data.get('fdv', 0),
                    'marketCap': token_data.get('marketCap', 0),
                    'pairCreatedAt': token_data.get('pairCreatedAt', '')
                }
                token_details_list.append(token_details)
            logger.info(f"fetch_token_details: Completed processing batch {i//30 + 1}.")
            sleep(1)  # avoid rate limiting
        except requests.RequestException as e:
            logger.error(f"fetch_token_details: Error fetching batch {batch_addresses}: {e}")
            sleep(1)

    token_details = pd.DataFrame(token_details_list)
    logger.info("fetch_token_details: Finished fetching all token details.")
    return token_details


# -------------------------------------------------------------------
# Main function with progress bars for transactions and tokens processing
def main():
    # Initialize configuration and database connections
    config = load_config()
    db_connection = DatabaseConnection(config.db)
    db_queries = DatabaseQueries(db_connection)

    try:
        logger.info("main: Starting main function execution.")

        # (1) Update token information (e.g., current prices)
        # Uncomment the following lines if you want to update tokens
        # logger.info("main: Calling update_tokens.main()...")
        # update_tokens_main()

        # (2) Fetch recent transactions (e.g., from the past 6 hours)
        logger.info("main: Fetching recent transactions...")
        transactions_df = db_queries.get_recent_transactions(n=1)
        logger.info(f"main: Fetched {len(transactions_df)} transactions.")

        # (3) Fetch token details once
        token_columns = [
            "open_input_mint", "open_output_mint", "close_input_mint", "close_output_mint"
        ]
        unique_tokens = pd.unique(transactions_df[token_columns].values.ravel())
        logger.info(f"main: Found {len(unique_tokens)} unique token addresses for details.")
        chain_id = "solana"
        api_url = "https://api.dexscreener.com"
        token_details_df = fetch_token_details(chain_id, unique_tokens, api_url)

        # (4) Add new columns for chosen tokens and their historical prices
        transactions_df["input_token"] = None
        transactions_df["output_token"] = None
        transactions_df["input_price"] = None
        transactions_df["output_price"] = None

        # (5) Use a cache to avoid duplicate API calls for the same (token, block_time) combination
        price_cache = {}
        logger.info("main: Processing transactions to fetch historical prices...")
        # Wrap the transactions iteration with tqdm for progress reporting
        for index, row in tqdm(transactions_df.iterrows(), total=len(transactions_df), desc="Processing transactions"):
            if pd.notnull(row["open_input_mint"]):
                input_token = row["open_input_mint"]
                output_token = row["open_output_mint"]
                block_time = row["open_block_time"]
            else:
                input_token = row["close_input_mint"]
                output_token = row["close_output_mint"]
                block_time = row["close_block_time"]

            transactions_df.at[index, "input_token"] = input_token
            transactions_df.at[index, "output_token"] = output_token

            # Fetch historical price for input token using caching
            key_input = (input_token, block_time)
            if key_input in price_cache:
                input_price = price_cache[key_input]
                logger.debug(f"main: Using cached price for input token '{input_token}' at {block_time}.")
            else:
                input_price = fetch_historical_price(input_token, block_time, config.birdeye.api_key)
                price_cache[key_input] = input_price
                logger.debug(f"main: Cached price {input_price} for input token '{input_token}' at {block_time}.")

            # Fetch historical price for output token using caching
            key_output = (output_token, block_time)
            if key_output in price_cache:
                output_price = price_cache[key_output]
                logger.debug(f"main: Using cached price for output token '{output_token}' at {block_time}.")
            else:
                output_price = fetch_historical_price(output_token, block_time, config.birdeye.api_key)
                price_cache[key_output] = output_price
                logger.debug(f"main: Cached price {output_price} for output token '{output_token}' at {block_time}.")

            transactions_df.at[index, "input_price"] = input_price
            transactions_df.at[index, "output_price"] = output_price

            sleep(0.2)  # Avoid rate limiting

        # (6) Calculate per-transaction dollar amounts:
        logger.info("main: Calculating per-transaction dollar values...")
        transactions_df["dollars_sold"] = (
                transactions_df["in_amount"].astype(float) *
                transactions_df["input_price"].astype(float)
        )
        transactions_df["dollars_bought"] = (
                transactions_df["out_amount"].astype(float) *
                transactions_df["output_price"].astype(float)
        )

        # (7) Aggregate dollar amounts per token:
        logger.info("main: Aggregating dollar values per token...")
        aggregated_sold = (
            transactions_df.groupby("input_token")["dollars_sold"]
            .sum()
            .reset_index()
            .rename(columns={"input_token": "token"})
        )
        aggregated_bought = (
            transactions_df.groupby("output_token")["dollars_bought"]
            .sum()
            .reset_index()
            .rename(columns={"output_token": "token"})
        )
        aggregated = pd.merge(aggregated_sold, aggregated_bought, on="token", how="outer")
        aggregated["dollars_sold"] = aggregated["dollars_sold"].fillna(0)
        aggregated["dollars_bought"] = aggregated["dollars_bought"].fillna(0)
        logger.info("main: Aggregated token values:")
        logger.info(aggregated)

        # (8) Merge aggregated data with token details
        logger.info("main: Merging aggregated data with token details...")
        final_df = pd.merge(aggregated, token_details_df, left_on="token", right_on="baseToken_address", how="left")

        # (9) Format dollars columns to use a comma as the decimal separator.
        logger.info("main: Formatting dollar values with comma as decimal separator...")
        final_df["dollars_sold"] = final_df["dollars_sold"].apply(lambda x: f"{x:.2f}".replace('.',','))
        final_df["dollars_bought"] = final_df["dollars_bought"].apply(lambda x: f"{x:.2f}".replace('.',','))

        # (10) Remove timezone information for Excel compatibility.
        for col in transactions_df.select_dtypes(include=["datetimetz"]).columns:
            transactions_df[col] = transactions_df[col].dt.tz_localize(None)
        for col in final_df.select_dtypes(include=["datetimetz"]).columns:
            final_df[col] = final_df[col].dt.tz_localize(None)

        # (11) Write the final merged output to an Excel file.
        logger.info("main: Writing final aggregated token details to Excel file...")
        final_df.to_excel("final_aggregated_token_details.xlsx", index=False)
        logger.info("main: Final Excel file written successfully.")
        print(final_df)

    except Exception as e:
        logger.error(f"main: An error occurred: {e}", exc_info=True)
    finally:
        db_connection.close()
        logger.info("main: Database connection closed.")

if __name__ == "__main__":
    main()


2025-02-05 18:06:27,773 - INFO - main: Starting main function execution.
2025-02-05 18:06:27,775 - INFO - main: Fetching recent transactions...
2025-02-05 18:06:27,776 - INFO - Fetching recent transactions for the past 1 hours.
2025-02-05 18:06:28,500 - INFO - Fetched 80 rows of transactions.
2025-02-05 18:06:28,502 - INFO - main: Fetched 80 transactions.
2025-02-05 18:06:28,503 - INFO - main: Found 60 unique token addresses for details.
2025-02-05 18:06:28,504 - INFO - fetch_token_details: Starting to fetch details for 60 tokens on chain 'solana'.
Processing token batches:   0%|          | 0/2 [00:00<?, ?batch/s]2025-02-05 18:06:28,507 - INFO - fetch_token_details: Processing batch 1 with tokens: ['EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', 'jupSoLaHXQiZZTSfEWMTRRgpnyFm8f6sZdosWBjx93v', '9bNUjxEvygayUE2ZRN5zh9Hhjh6cN6GPK4zoHzXXpump', '2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv', '3ufbSaVQSGeoCbgVp45YYYWKygbYFXAUZ7n7AP4CUWg6', 'So11111111111111111111111111111111111111112', '63LfD

In [1]:
import logging
import requests
import pandas as pd
import time
from time import sleep
from tqdm import tqdm  # progress bar package
from processors.update_tokens import main as update_tokens_main
import sys
from db.connection import DatabaseConnection
from db.queries import DatabaseQueries
from config import load_config
import random

# Configure logging globally
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# -------------------------------------------------------------------
def fetch_historical_price(token_mint, api_key):
    """
    Fetch the historical price data for a given token using the Birdeye API endpoint
    /defi/history_price. The API call requests the price history from:
        time_from = now - 188700 (seconds)
        time_to   = now

    The API is expected to return a JSON with a structure like:
    {
      "data": {
        "items": [
          {
            "address": "ZYDWbrsVbaE83FVXvdHoUVcKRrDN9S2Jskyhsuypump",
            "unixTime": 1738942800,
            "value": 0.00000610584940167204
          },
          ...
        ]
      },
      "success": true
    }

    Parameters:
        token_mint (str): The token’s mint address.
        api_key (str): The Birdeye API key.

    Returns:
        pd.DataFrame: A DataFrame containing the historical price snapshots (sorted by unixTime)
                      if the call is successful, otherwise None.
    """
    now = int(time.time())
    time_from = now - 188700  # 188700 seconds before now
    url = f"https://public-api.birdeye.so/defi/history_price?address={token_mint}&address_type=token&type=5m&time_from={time_from}&time_to={now}"
    logger.info(f"fetch_historical_price: Requesting history for token '{token_mint}' from {time_from} to {now}.")
    headers = {
        "accept": "application/json",
        "x-chain": "solana",
        "X-API-KEY": api_key
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        if data.get("success") is True:
            items = data.get("data", {}).get("items", [])
            if items:
                df = pd.DataFrame(items)
                df = df.sort_values("unixTime")
                logger.info(f"fetch_historical_price: Retrieved {len(df)} price snapshots for token '{token_mint}'.")
                return df
            else:
                logger.error(f"fetch_historical_price: No items found in API response for token '{token_mint}'. Response: {data}")
                return None
        else:
            logger.error(f"fetch_historical_price: API returned error for token '{token_mint}'. Response: {data}")
            return None
    except requests.RequestException as e:
        logger.error(f"fetch_historical_price: Request error for token '{token_mint}': {e}")
        return None

# -------------------------------------------------------------------
def fetch_token_details(chain_id, token_addresses, api_url):
    """
    Fetch token details from the Dexscreener API in batches and return a DataFrame.
    (This function remains unchanged.)
    """
    logger.info(f"fetch_token_details: Starting to fetch details for {len(token_addresses)} tokens on chain '{chain_id}'.")
    token_details_list = []
    token_addresses = [address for address in token_addresses if address is not None]
    total_batches = (len(token_addresses) + 29) // 30
    for i in tqdm(range(0, len(token_addresses), 30), desc='Processing token batches', unit='batch', total=total_batches):
        batch_addresses = token_addresses[i:i+30]
        logger.info(f"fetch_token_details: Processing batch {i//30 + 1} with tokens: {batch_addresses}")
        try:
            response = requests.get(f"{api_url}/tokens/v1/{chain_id}/{','.join(batch_addresses)}")
            response.raise_for_status()
            data = response.json()
            for token_data in data:
                token_details = {
                    'chainId': token_data.get('chainId', ''),
                    'dexId': token_data.get('dexId', ''),
                    'url': token_data.get('url', ''),
                    'pairAddress': token_data.get('pairAddress', ''),
                    'labels': token_data.get('labels', []),
                    'baseToken_address': token_data.get('baseToken', {}).get('address', ''),
                    'baseToken_name': token_data.get('baseToken', {}).get('name', ''),
                    'baseToken_symbol': token_data.get('baseToken', {}).get('symbol', ''),
                    'quoteToken_address': token_data.get('quoteToken', {}).get('address', ''),
                    'quoteToken_name': token_data.get('quoteToken', {}).get('name', ''),
                    'quoteToken_symbol': token_data.get('quoteToken', {}).get('symbol', ''),
                    'priceNative': token_data.get('priceNative', 0.0),
                    'priceUsd': token_data.get('priceUsd', 0.0),
                    'txns_m5_buys': token_data.get('txns', {}).get('m5', {}).get('buys', 0),
                    'txns_m5_sells': token_data.get('txns', {}).get('m5', {}).get('sells', 0),
                    'txns_h1_buys': token_data.get('txns', {}).get('h1', {}).get('buys', 0),
                    'txns_h1_sells': token_data.get('txns', {}).get('h1', {}).get('sells', 0),
                    'txns_h6_buys': token_data.get('txns', {}).get('h6', {}).get('buys', 0),
                    'txns_h6_sells': token_data.get('txns', {}).get('h6', {}).get('sells', 0),
                    'txns_h24_buys': token_data.get('txns', {}).get('h24', {}).get('buys', 0),
                    'txns_h24_sells': token_data.get('txns', {}).get('h24', {}).get('sells', 0),
                    'volume_h24': token_data.get('volume', {}).get('h24', 0.0),
                    'volume_h6': token_data.get('volume', {}).get('h6', 0.0),
                    'volume_h1': token_data.get('volume', {}).get('h1', 0.0),
                    'volume_m5': token_data.get('volume', {}).get('m5', 0.0),
                    'priceChange_m5': token_data.get('priceChange', {}).get('m5', 0.0),
                    'priceChange_h1': token_data.get('priceChange', {}).get('h1', 0.0),
                    'priceChange_h6': token_data.get('priceChange', {}).get('h6', 0.0),
                    'priceChange_h24': token_data.get('priceChange', {}).get('h24', 0.0),
                    'liquidity_usd': token_data.get('liquidity', {}).get('usd', 0.0),
                    'liquidity_base': token_data.get('liquidity', {}).get('base', 0.0),
                    'liquidity_quote': token_data.get('liquidity', {}).get('quote', 0.0),
                    'fdv': token_data.get('fdv', 0),
                    'marketCap': token_data.get('marketCap', 0),
                    'pairCreatedAt': token_data.get('pairCreatedAt', '')
                }
                token_details_list.append(token_details)
            logger.info(f"fetch_token_details: Completed processing batch {i//30 + 1}.")
            sleep(1)  # avoid rate limiting
        except requests.RequestException as e:
            logger.error(f"fetch_token_details: Error fetching batch {batch_addresses}: {e}")
            sleep(1)
    token_details = pd.DataFrame(token_details_list)
    logger.info("fetch_token_details: Finished fetching all token details.")
    return token_details

# -------------------------------------------------------------------
def get_closest_price(token, block_time, token_history_data):
    """
    Given a token and a block_time, look up the historical price data for that token
    (as returned by fetch_historical_price) and return the price (i.e. 'value') from the
    snapshot with the smallest time difference to block_time.
    """
    df = token_history_data.get(token)
    if df is None or df.empty:
        logger.warning(f"get_closest_price: No historical data available for token {token}.")
        return None
    # Calculate the absolute time difference between block_time and all snapshots
    time_diffs = (df["unixTime"] - block_time).abs()
    closest_idx = time_diffs.idxmin()
    closest_price = df.loc[closest_idx, "value"]
    return closest_price

# -------------------------------------------------------------------
def main():
    # Initialize configuration and database connections
    config = load_config()
    db_connection = DatabaseConnection(config.db)
    db_queries = DatabaseQueries(db_connection)

    try:
        logger.info("main: Starting main function execution.")

        # (1) Optionally update token information
        # logger.info("main: Calling update_tokens.main()...")
        # update_tokens_main()

        # (2) Fetch recent transactions (e.g., from the past 6 hours)
        logger.info("main: Fetching recent transactions...")
        transactions_df = db_queries.get_recent_transactions(n=3)
        logger.info(f"main: Fetched {len(transactions_df)} transactions.")

        # (3) Fetch token details once
        token_columns = ["open_input_mint", "open_output_mint", "close_input_mint", "close_output_mint"]
        unique_tokens = pd.unique(transactions_df[token_columns].values.ravel())
        logger.info(f"main: Found {len(unique_tokens)} unique token addresses for details.")
        chain_id = "solana"
        api_url = "https://api.dexscreener.com"
        token_details_df = fetch_token_details(chain_id, unique_tokens, api_url)

        # (4) Add new columns for tokens and their historical prices
        transactions_df["input_token"] = None
        transactions_df["output_token"] = None
        transactions_df["input_price"] = None
        transactions_df["output_price"] = None

        # (5) For each distinct token, fetch its historical price snapshots once
        logger.info("main: Fetching historical price data for each unique token...")
        token_history_data = {}
        for token in tqdm(unique_tokens, desc="Fetching token historical prices", unit="token"):
            df_history = fetch_historical_price(token, config.birdeye.api_key)
            if df_history is not None:
                token_history_data[token] = df_history
            else:
                logger.warning(f"main: No historical data found for token {token}.")
            sleep(0.2)  # To avoid rate limiting

        # (6) Process each transaction to look up the closest historical price for both input and output tokens
        logger.info("main: Processing transactions to calculate historical prices...")
        for index, row in tqdm(transactions_df.iterrows(), total=len(transactions_df), desc="Processing transactions"):
            if pd.notnull(row["open_input_mint"]):
                input_token = row["open_input_mint"]
                output_token = row["open_output_mint"]
                block_time = row["open_block_time"]
            else:
                input_token = row["close_input_mint"]
                output_token = row["close_output_mint"]
                block_time = row["close_block_time"]

            transactions_df.at[index, "input_token"] = input_token
            transactions_df.at[index, "output_token"] = output_token

            # Look up the closest price snapshot for each token based on block_time
            input_price = get_closest_price(input_token, block_time, token_history_data)
            output_price = get_closest_price(output_token, block_time, token_history_data)

            transactions_df.at[index, "input_price"] = input_price
            transactions_df.at[index, "output_price"] = output_price

        # (7) Calculate per-transaction dollar amounts
        logger.info("main: Calculating per-transaction dollar values...")
        transactions_df["dollars_sold"] = (
                transactions_df["in_amount"].astype(float) *
                transactions_df["input_price"].astype(float)
        )
        transactions_df["dollars_bought"] = (
                transactions_df["out_amount"].astype(float) *
                transactions_df["output_price"].astype(float)
        )

        # (8) Aggregate dollar amounts per token
        logger.info("main: Aggregating dollar values per token...")
        aggregated_sold = (
            transactions_df.groupby("input_token")["dollars_sold"]
            .sum()
            .reset_index()
            .rename(columns={"input_token": "token"})
        )
        aggregated_bought = (
            transactions_df.groupby("output_token")["dollars_bought"]
            .sum()
            .reset_index()
            .rename(columns={"output_token": "token"})
        )
        aggregated = pd.merge(aggregated_sold, aggregated_bought, on="token", how="outer")
        aggregated["dollars_sold"] = aggregated["dollars_sold"].fillna(0)
        aggregated["dollars_bought"] = aggregated["dollars_bought"].fillna(0)
        logger.info("main: Aggregated token values:")
        logger.info(aggregated)

        # (9) Merge aggregated data with token details
        logger.info("main: Merging aggregated data with token details...")
        final_df = pd.merge(aggregated, token_details_df, left_on="token", right_on="baseToken_address", how="left")

        # (10) Format dollars columns to use a comma as the decimal separator
        logger.info("main: Formatting dollar values with comma as decimal separator...")
        final_df["dollars_sold"] = final_df["dollars_sold"].apply(lambda x: f"{x:.2f}".replace('.',','))
        final_df["dollars_bought"] = final_df["dollars_bought"].apply(lambda x: f"{x:.2f}".replace('.',','))

        # (11) Remove timezone information for Excel compatibility
        for col in transactions_df.select_dtypes(include=["datetimetz"]).columns:
            transactions_df[col] = transactions_df[col].dt.tz_localize(None)
        for col in final_df.select_dtypes(include=["datetimetz"]).columns:
            final_df[col] = final_df[col].dt.tz_localize(None)

        # (12) Write the final merged output to an Excel file.
        logger.info("main: Writing final aggregated token details to Excel file...")
        final_df.to_excel("final_aggregated_token_details.xlsx", index=False)
        logger.info("main: Final Excel file written successfully.")
        print(final_df)

    except Exception as e:
        logger.error(f"main: An error occurred: {e}", exc_info=True)
    finally:
        db_connection.close()
        logger.info("main: Database connection closed.")

if __name__ == "__main__":
    main()


INFO:__main__:main: Starting main function execution.
INFO:__main__:main: Fetching recent transactions...
INFO:db.queries:Fetching recent transactions for the past 3 hours.
INFO:db.queries:Fetched 208 rows of transactions.
INFO:__main__:main: Fetched 208 transactions.
INFO:__main__:main: Found 77 unique token addresses for details.
INFO:__main__:fetch_token_details: Starting to fetch details for 77 tokens on chain 'solana'.
Processing token batches:   0%|          | 0/3 [00:00<?, ?batch/s]INFO:__main__:fetch_token_details: Processing batch 1 with tokens: ['AT7RRrFhBU1Dw1WghdgAqeNKNXKomDFXm77owQgppump', 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', 'So11111111111111111111111111111111111111112', 'EqeEBGHQhQy6SqeaJcnqAsNs3qaG19sdF89Xsarpump', '27G8MtK7VtTcCHkpASjSDdkWWYfoqT6ggEuKidVJidD4', '9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump', '5oVNBeEEQvYi1cX3ir8Dx5n1P7pdxydbGF2X4TxVusJm', 'Grass7B4RdKfBCjTKgSqnXkqjwiGvQyFbuSCUJr3XXjs', 'FFgfStKwuF3DSxEeogA69FNkPrkb7XDA5Tw29TBEpump', 'CRAMvzD

                                           token dollars_sold dollars_bought  \
0   22Xeo6diWfJrScaoVFzgkwzrCByPukK45fdwkJyrpump         0,00        3930,97   
1   27G8MtK7VtTcCHkpASjSDdkWWYfoqT6ggEuKidVJidD4      2936,53        2614,84   
2   2KchKijPuwnwC92LPWVjFjRwB3WxKtzx9bbXZ7kRpump      2242,65           0,00   
3   2RBko3xoz56aH69isQMUpzZd9NYHahhwC23A5F3Spkin         0,00         329,75   
4   2RuDRx9RAcXrSoLupeMLGuBay6w5Q1nUrdPySjA3pump       277,27           0,00   
..                                           ...          ...            ...   
72   jUfwXi6BWechD13bzp1R1h2AR3bSKuGkJd5qJmppump      1693,16           0,00   
73   jupSoLaHXQiZZTSfEWMTRRgpnyFm8f6sZdosWBjx93v         0,00        2034,05   
74   nosXBVoaCTtYdLvKY6Csb4AC8JCdQKKAaWYtx2ZMoo7      6102,52           0,00   
75   pepo1CFNU2RXf7yXX7HNXazXwxsq8WrPvDHpHriwoLY         0,00         971,96   
76   whispF7G9DHaojYHe2cdhRX5EMJzGBdqq7R57kL6inL         0,00       43362,65   

   chainId    dexId                    

In [3]:
import logging
import requests
import pandas as pd
import time
from time import sleep
from tqdm import tqdm  # progress bar package
from processors.update_tokens import main as update_tokens_main
import sys
from db.connection import DatabaseConnection
from db.queries import DatabaseQueries
from config import load_config
import random
import os
from datetime import datetime

# Configure logging globally
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# -------------------------------------------------------------------
def fetch_historical_price(token_mint, api_key):
    """
    Fetch the historical price data for a given token using the Birdeye API endpoint
    /defi/history_price. The API call requests the price history from:
        time_from = now - 188700 (seconds)
        time_to   = now

    Returns a DataFrame with historical snapshots sorted by unixTime or None if not found.
    """
    now = int(time.time())
    time_from = now - 188700  # 188700 seconds before now
    url = f"https://public-api.birdeye.so/defi/history_price?address={token_mint}&address_type=token&type=5m&time_from={time_from}&time_to={now}"
    logger.info(f"fetch_historical_price: Requesting history for token '{token_mint}' from {time_from} to {now}.")
    headers = {
        "accept": "application/json",
        "x-chain": "solana",
        "X-API-KEY": api_key
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        if data.get("success") is True:
            items = data.get("data", {}).get("items", [])
            if items:
                df = pd.DataFrame(items)
                df = df.sort_values("unixTime")
                logger.info(f"fetch_historical_price: Retrieved {len(df)} price snapshots for token '{token_mint}'.")
                return df
            else:
                logger.error(f"fetch_historical_price: No items found in API response for token '{token_mint}'. Response: {data}")
                return None
        else:
            logger.error(f"fetch_historical_price: API returned error for token '{token_mint}'. Response: {data}")
            return None
    except requests.RequestException as e:
        logger.error(f"fetch_historical_price: Request error for token '{token_mint}': {e}")
        return None

# -------------------------------------------------------------------
def fetch_token_details(chain_id, token_addresses, api_key):
    """
    Fetch token details from the Birdeye token overview endpoint.
    For each token address, it sends a GET request to:
      https://public-api.birdeye.so/defi/token_overview?address={token}
    The chain is specified in the request headers.

    Returns a DataFrame containing the details for all tokens.
    """
    logger.info(f"fetch_token_details: Starting to fetch details for {len(token_addresses)} tokens on chain '{chain_id}' using Birdeye API.")
    token_details_list = []
    # Filter out any None values
    token_addresses = [address for address in token_addresses if address is not None]
    for token in tqdm(token_addresses, desc='Fetching token details', unit='token'):
        url = f"https://public-api.birdeye.so/defi/token_overview?address={token}"
        headers = {
            "accept": "application/json",
            "x-chain": chain_id,
            "X-API-KEY": api_key
        }
        logger.info(f"fetch_token_details: Fetching details for token: {token}")
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            if data.get("success") is True:
                token_data = data.get("data", {})
                # Ensure the 'address' field exists for merging later.
                if "address" not in token_data:
                    token_data["address"] = token
                token_details_list.append(token_data)
                logger.info(f"fetch_token_details: Retrieved details for token: {token}")
            else:
                logger.error(f"fetch_token_details: API returned error for token {token}. Response: {data}")
            sleep(0.2)  # avoid rate limiting
        except requests.RequestException as e:
            logger.error(f"fetch_token_details: Request error for token {token}: {e}")
            sleep(0.2)
    token_details_df = pd.DataFrame(token_details_list)
    logger.info("fetch_token_details: Finished fetching all token details.")
    return token_details_df

# -------------------------------------------------------------------
def get_closest_price(token, block_time, token_history_data):
    """
    Given a token and a block_time, look up the historical price data for that token
    (as returned by fetch_historical_price) and return the price ('value') from the
    snapshot with the smallest time difference to block_time.
    """
    df = token_history_data.get(token)
    if df is None or df.empty:
        logger.warning(f"get_closest_price: No historical data available for token {token}.")
        return None
    # Calculate the absolute time difference between block_time and all snapshots
    time_diffs = (df["unixTime"] - block_time).abs()
    closest_idx = time_diffs.idxmin()
    closest_price = df.loc[closest_idx, "value"]
    return closest_price

# -------------------------------------------------------------------
def main():
    # Initialize configuration and database connections
    config = load_config()
    db_connection = DatabaseConnection(config.db)
    db_queries = DatabaseQueries(db_connection)

    try:
        logger.info("main: Starting main function execution.")

        # (1) Optionally update token information
        # logger.info("main: Calling update_tokens.main()...")
        # update_tokens_main()

        # (2) Fetch recent transactions (e.g., from the past 6 hours)
        logger.info("main: Fetching recent transactions...")
        transactions_df = db_queries.get_recent_transactions(n=3)
        logger.info(f"main: Fetched {len(transactions_df)} transactions.")

        # (3) Fetch token details once using the Birdeye API
        token_columns = ["open_input_mint", "open_output_mint", "close_input_mint", "close_output_mint"]
        unique_tokens = pd.unique(transactions_df[token_columns].values.ravel())
        logger.info(f"main: Found {len(unique_tokens)} unique token addresses for details.")
        chain_id = "solana"
        token_details_df = fetch_token_details(chain_id, unique_tokens, config.birdeye.api_key)

        # (4) Add new columns for tokens and their historical prices
        transactions_df["input_token"] = None
        transactions_df["output_token"] = None
        transactions_df["input_price"] = None
        transactions_df["output_price"] = None

        # (5) For each distinct token, fetch its historical price snapshots once
        logger.info("main: Fetching historical price data for each unique token...")
        token_history_data = {}
        for token in tqdm(unique_tokens, desc="Fetching token historical prices", unit="token"):
            df_history = fetch_historical_price(token, config.birdeye.api_key)
            if df_history is not None:
                token_history_data[token] = df_history
            else:
                logger.warning(f"main: No historical data found for token {token}.")
            sleep(0.2)  # To avoid rate limiting

        # (6) Process each transaction to look up the closest historical price for both input and output tokens
        logger.info("main: Processing transactions to calculate historical prices...")
        for index, row in tqdm(transactions_df.iterrows(), total=len(transactions_df), desc="Processing transactions"):
            if pd.notnull(row["open_input_mint"]):
                input_token = row["open_input_mint"]
                output_token = row["open_output_mint"]
                block_time = row["open_block_time"]
            else:
                input_token = row["close_input_mint"]
                output_token = row["close_output_mint"]
                block_time = row["close_block_time"]

            transactions_df.at[index, "input_token"] = input_token
            transactions_df.at[index, "output_token"] = output_token

            # Look up the closest price snapshot for each token based on block_time
            input_price = get_closest_price(input_token, block_time, token_history_data)
            output_price = get_closest_price(output_token, block_time, token_history_data)

            transactions_df.at[index, "input_price"] = input_price
            transactions_df.at[index, "output_price"] = output_price

        # (7) Calculate per-transaction dollar amounts
        logger.info("main: Calculating per-transaction dollar values...")
        transactions_df["dollars_sold"] = (
                transactions_df["in_amount"].astype(float) *
                transactions_df["input_price"].astype(float)
        )
        transactions_df["dollars_bought"] = (
                transactions_df["out_amount"].astype(float) *
                transactions_df["output_price"].astype(float)
        )

        # (8) Aggregate dollar amounts per token
        logger.info("main: Aggregating dollar values per token...")
        aggregated_sold = (
            transactions_df.groupby("input_token")["dollars_sold"]
            .sum()
            .reset_index()
            .rename(columns={"input_token": "token"})
        )
        aggregated_bought = (
            transactions_df.groupby("output_token")["dollars_bought"]
            .sum()
            .reset_index()
            .rename(columns={"output_token": "token"})
        )
        aggregated = pd.merge(aggregated_sold, aggregated_bought, on="token", how="outer")
        aggregated["dollars_sold"] = aggregated["dollars_sold"].fillna(0)
        aggregated["dollars_bought"] = aggregated["dollars_bought"].fillna(0)
        logger.info("main: Aggregated token values:")
        logger.info(aggregated)

        # (9) Merge aggregated data with token details
        # We merge on the token address field ("address") from the Birdeye API data.
        logger.info("main: Merging aggregated data with token details...")
        final_df = pd.merge(aggregated, token_details_df, left_on="token", right_on="address", how="left")

        # (10) Format dollars columns to use a comma as the decimal separator
        logger.info("main: Formatting dollar values with comma as decimal separator...")
        final_df["dollars_sold"] = final_df["dollars_sold"].apply(lambda x: f"{x:.2f}".replace('.',','))
        final_df["dollars_bought"] = final_df["dollars_bought"].apply(lambda x: f"{x:.2f}".replace('.',','))

        # (11) Remove timezone information for Excel compatibility
        for col in transactions_df.select_dtypes(include=["datetimetz"]).columns:
            transactions_df[col] = transactions_df[col].dt.tz_localize(None)
        for col in final_df.select_dtypes(include=["datetimetz"]).columns:
            final_df[col] = final_df[col].dt.tz_localize(None)

        # (12) Create a folder to store generated Excel files (if not exists) and write the final merged output to an Excel file.
        output_folder = "../data/"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            logger.info(f"main: Created output folder '{output_folder}'.")

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        excel_filename = f"final_aggregated_token_details_{timestamp}.xlsx"
        output_path = os.path.join(output_folder, excel_filename)

        logger.info(f"main: Writing final aggregated token details to Excel file at '{output_path}'...")
        final_df.to_excel(output_path, index=False)
        logger.info("main: Final Excel file written successfully.")
        print(final_df)

    except Exception as e:
        logger.error(f"main: An error occurred: {e}", exc_info=True)
    finally:
        db_connection.close()
        logger.info("main: Database connection closed.")

if __name__ == "__main__":
    main()


INFO:__main__:main: Starting main function execution.
INFO:__main__:main: Fetching recent transactions...
INFO:db.queries:Fetching recent transactions for the past 3 hours.
INFO:db.queries:Fetched 232 rows of transactions.
INFO:__main__:main: Fetched 232 transactions.
INFO:__main__:main: Found 84 unique token addresses for details.
INFO:__main__:fetch_token_details: Starting to fetch details for 84 tokens on chain 'solana' using Birdeye API.
Fetching token details:   0%|          | 0/84 [00:00<?, ?token/s]INFO:__main__:fetch_token_details: Fetching details for token: AT7RRrFhBU1Dw1WghdgAqeNKNXKomDFXm77owQgppump
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): public-api.birdeye.so:443
DEBUG:urllib3.connectionpool:https://public-api.birdeye.so:443 "GET /defi/token_overview?address=AT7RRrFhBU1Dw1WghdgAqeNKNXKomDFXm77owQgppump HTTP/1.1" 200 None
INFO:__main__:fetch_token_details: Retrieved details for token: AT7RRrFhBU1Dw1WghdgAqeNKNXKomDFXm77owQgppump
Fetching token detail

                                           token dollars_sold dollars_bought  \
0   27G8MtK7VtTcCHkpASjSDdkWWYfoqT6ggEuKidVJidD4      5167,17        5050,65   
1   2KchKijPuwnwC92LPWVjFjRwB3WxKtzx9bbXZ7kRpump      2242,65           0,00   
2   2RBko3xoz56aH69isQMUpzZd9NYHahhwC23A5F3Spkin         0,00         329,75   
3   2eXamy7t3kvKhfV6aJ6Uwe3eh8cuREFcTKs1mFKZpump         0,00       12536,42   
4   2ru87k7yAZnDRsnqVpgJYETFgqVApuBcwB2xDb19pump     16535,50           0,00   
..                                           ...          ...            ...   
79   ZEXy1pqteRu3n13kdyh4LwPQknkFk3GzmMYMuNadWPo         0,00         990,05   
80   h5NciPdMZ5QCB5BYETJMYBMpVx9ZuitR6HcVjyBhood     20415,07           0,00   
81   jUfwXi6BWechD13bzp1R1h2AR3bSKuGkJd5qJmppump     29164,15        4750,60   
82   jupSoLaHXQiZZTSfEWMTRRgpnyFm8f6sZdosWBjx93v         0,00        2034,05   
83   whispF7G9DHaojYHe2cdhRX5EMJzGBdqq7R57kL6inL         0,00       48324,74   

                                       